# Segmentação Semântica para Processamento de Documentos

## Visão Geral

Este código implementa uma abordagem de segmentação semântica para processar e recuperar informações de documentos PDF, [proposta inicialmente por Greg Kamradt](https://youtu.be/8OJC21T2SL4?t=1933) e posteriormente [implementada no LangChain](https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/). Diferentemente dos métodos tradicionais que dividem o texto com base em contagens fixas de caracteres ou palavras, a segmentação semântica visa criar segmentos de texto mais significativos e contextualmente conscientes.

## Motivação

Os métodos tradicionais de divisão de texto frequentemente quebram documentos em pontos arbitrários, potencialmente interrompendo o fluxo de informações e o contexto. A segmentação semântica aborda esse problema tentando dividir o texto em pontos de quebra mais naturais, preservando a coerência semântica dentro de cada segmento.

## Componentes Principais

1. Processamento de PDF e extração de texto
2. Segmentação semântica usando o `SemanticChunker` do LangChain
3. Criação de um armazenamento de vetores usando FAISS e embeddings da OpenAI
4. Configuração de um recuperador para consultar os documentos processados

## Detalhes do Método

### Pré-processamento do Documento

1. O PDF é lido e convertido em uma string usando uma função personalizada `read_pdf_to_string`.

### Segmentação Semântica

1. Utiliza o `SemanticChunker` do LangChain com embeddings da OpenAI.
2. Três tipos de pontos de quebra estão disponíveis:
   - 'percentile': Divide em diferenças maiores que o percentil X.
   - 'standard_deviation': Divide em diferenças maiores que X desvios padrão.
   - 'interquartile': Usa a distância interquartil para determinar os pontos de divisão.
3. Nesta implementação, o método 'percentile' é usado com um limite de 90.

### Criação do Armazenamento de Vetores

1. Os embeddings da OpenAI são usados para criar representações vetoriais dos segmentos semânticos.
2. Um armazenamento de vetores FAISS é criado a partir desses embeddings para uma busca de similaridade eficiente.

### Configuração do Recuperador

1. Um recuperador é configurado para buscar os 2 segmentos mais relevantes para uma determinada consulta.

## Características Principais

1. Divisão Consciente do Contexto: Tenta manter a coerência semântica dentro dos segmentos.
2. Configuração Flexível: Permite diferentes tipos de pontos de quebra e limites.
3. Integração com Ferramentas Avançadas de NLP: Usa embeddings da OpenAI tanto para a segmentação quanto para a recuperação.

## Benefícios dessa Abordagem

1. Coerência Melhorada: Os segmentos têm maior probabilidade de conter pensamentos ou ideias completas.
2. Melhor Relevância na Recuperação: Ao preservar o contexto, a precisão da recuperação pode ser aprimorada.
3. Adaptabilidade: O método de segmentação pode ser ajustado com base na natureza dos documentos e nas necessidades de recuperação.
4. Potencial para Melhor Compreensão: Modelos de linguagem ou tarefas subsequentes podem ter um desempenho melhor com segmentos de texto mais coerentes.

## Detalhes de Implementação

1. Usa embeddings da OpenAI tanto para o processo de segmentação semântica quanto para as representações vetoriais finais.
2. Emprega FAISS para criar um índice pesquisável eficiente dos segmentos.
3. O recuperador é configurado para retornar os 2 segmentos mais relevantes, que podem ser ajustados conforme necessário.

## Exemplo de Uso

O código inclui uma consulta de teste: "Qual é a principal causa das mudanças climáticas?". Isso demonstra como o sistema de segmentação semântica e recuperação pode ser usado para encontrar informações relevantes no documento processado.

## Conclusão

A segmentação semântica representa uma abordagem avançada para o processamento de documentos em sistemas de recuperação. Ao tentar manter a coerência semântica dentro dos segmentos de texto, ela tem o potencial de melhorar a qualidade das informações recuperadas e aprimorar o desempenho de tarefas subsequentes de NLP. Essa técnica é particularmente valiosa para processar documentos longos e complexos, onde a manutenção do contexto é crucial, como artigos científicos, documentos jurídicos ou relatórios abrangentes.

In [ ]:
import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Adiciona o diretório pai ao caminho, já que trabalhamos com notebooks
from helper_functions import *
from evaluation.evalute_rag import *

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Carrega variáveis de ambiente de um arquivo .env
load_dotenv()

# Define a variável de ambiente da chave da API da OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Definir caminho do arquivo

In [ ]:
path = "../data/Understanding_Climate_Change.pdf"

### Ler PDF para string

In [ ]:
content = read_pdf_to_string(path)

### Tipos de pontos de quebra:
* 'percentile': todas as diferenças entre as frases são calculadas, e então qualquer diferença maior que o percentil X é dividida.
* 'standard_deviation': qualquer diferença maior que X desvios padrão é dividida.
* 'interquartile': a distância interquartil é usada para dividir os segmentos.

In [ ]:
text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # escolhe quais embeddings, tipo de ponto de quebra e limite usar

### Dividir o texto original em segmentos semânticos

In [ ]:
docs = text_splitter.create_documents([content])

### Criar armazenamento de vetores e recuperador

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

### Testar o recuperador

In [ ]:
test_query = "Qual é a principal causa das mudanças climáticas?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)